# Importing the library

In [1]:
USE {
    repositories {
        maven {
            url = "https://maven.tryformation.com/releases"
        }
    }

    dependencies {
        implementation("com.github.jillesvangurp:json-dsl-jvm:1.2.2")
    }
}

Some imports that we need below

In [2]:
import com.jillesvangurp.jsondsl.*
import kotlin.reflect.KProperty

# Hello World

In [3]:
class HelloDsl : JsonDsl() {
  // adds a string property that the user can assign
  var message by property<String>()
}

JsonDsl uses property delegation to allow you to add typesafe properties. The delegated property stores it's value in the `Map`.

In [4]:
// a helper function to create HelloDsl instances
fun hello(block: HelloDsl.() -> Unit): HelloDsl {
  return HelloDsl().apply(block)
}

## Making it say hi

So now we can create some json from Kotlin

In [5]:
val ohai = hello {
  message = "Hello world"
}
// json is an extension function on JsonDsl that serializes everything
ohai.json(pretty = true)

{
  "message": "Hello world"
}

In [6]:
ohai.json()

{"message":"Hello world"}

It's just a MutableMap, so you can just add whatever you need with a simple put.

In [7]:
ohai["foo"] = "bar"
ohai.json(pretty = true)

{
  "message": "Hello world",
  "foo": "bar"
}

# Serialize Any?

JsonDsl is a `MutableMap<String, Any?>`. So it can have type safe properties for any Kotlin type. Including `Any?`

In [8]:
class AnyTypeDsl : JsonDsl() {
  var intVal by property<Int>()
  var boolVal by property<Boolean>()
  var doubleVal by property<Double>()
  var arrayVal by property<Array<String>>()
  var listVal by property<List<String>>()
  var mapVal by property<Map<String, String>>()
  var idontknow by property<Any>()
}

In [9]:
val anyDsl = AnyTypeDsl().apply {
  intVal = 1
  boolVal = true
  doubleVal = PI
  arrayVal = arrayOf("hello", "world")
  listVal = listOf("1", "2")
  mapVal = mapOf(
    "Key" to "Value"
  )

  // The Any type is a bit of free for all
  idontknow = mapOf(
    "arrays" to arrayOf(
      1, 2, "3", 4.0,
      mapOf("this" to "is valid JSON")
    ),
    "sequences" to sequenceOf(1,"2",3.0)
  )
}
    
anyDsl.json(true)

{
  "int_val": 1,
  "bool_val": true,
  "double_val": 3.141592653589793,
  "array_val": [
    "hello", 
    "world"
  ],
  "list_val": [
    "1", 
    "2"
  ],
  "map_val": {
    "Key": "Value"
  },
  "idontknow": {
    "arrays": [
      1, 
      2, 
      "3", 
      4.0, 
      {
        "this": "is valid JSON"
      }
    ],
    "sequences": [
      1, 
      "2", 
      3.0
    ]
  }
}

## toString() as a fallback

You don't have to create sub classes and you can just use `withJsonDsl` to quickly create some JsonDsl instance

In [10]:
data class FooBar(val foo:String="foo", val bar: String="bar") {
    override fun toString() = "$foo $bar"
}

println(withJsonDsl {
  this["foo"]=FooBar()
})

{
  "foo": "foo bar"
}


# YAML

There's a serializer for YAML too. Works the same way. Except it produces YAML.

In [11]:
anyDsl.yaml()

---
int_val: 1
bool_val: true
double_val: 3.141592653589793
array_val: 
  - hello
  - world
list_val: 
  - "1"
  - "2"
map_val: 
  Key: Value
idontknow: 
  arrays: 
    - 1
    - 2
    - "3"
    - 4.0
    - 
      this: "is valid JSON"
  sequences: 
    - 1
    - "2"
    - 3.0

Multi line strings work too in YAML

In [12]:
hello {
    message = """
        Hello
        world
    """.trimIndent()
}.yaml()

---
message: |
  Hello
  world

# Manipulating the map directly & Raw Json

In [13]:
val dsl = AnyTypeDsl().apply {
  // nicely typed.
  intVal = 42

  this["bar"] = "foo"
  this["going_off_script"] = listOf(
    AnyTypeDsl().apply {
      intVal = 666
      this["anything"] = "is possible"
    },
    42
  )
  this["inline_json"] = RawJson("""
    {
      "if":"you need to",
      "you":"can even add json in string form",
      "RawJson":"is a value class"
    }
  """.trimIndent())
}
println(dsl.json(true))

{
  "int_val": 42,
  "bar": "foo",
  "going_off_script": [
    {
      "int_val": 666,
      "anything": "is possible"
    }, 
    42
  ],
  "inline_json": {
  "if":"you need to",
  "you":"can even add json in string form",
  "RawJson":"is a value class"
}
}


# Snake Casing and custom names

In [14]:
class SnakeCaseDsl : JsonDsl(
  // actually the default
  namingConvention = PropertyNamingConvention.ConvertToSnakeCase
) {
  // this will be snake cased
  var camelCase by property<Boolean>()
  var mySize by property<Int>(
    customPropertyName = "size"
  )
  var myVal by property<String>(
    customPropertyName = "val"
  )
  // explicitly set name and provide a default
  var m by property(
    customPropertyName = "meaning_of_life",
    defaultValue = 42
  )
}

In [15]:
val dsl = SnakeCaseDsl().apply {
  camelCase = true
  mySize = Int.MAX_VALUE
  myVal = "hello"
}
dsl.json(true)

{
  "meaning_of_life": 42,
  "camel_case": true,
  "size": 2147483647,
  "val": "hello"
}

# Custom Values

In [16]:
enum class Grades(override val value: Double) : CustomValue<Double> {
  Excellent(7.0),
  Pass(5.51),
  Fail(3.0)
}

In [17]:
withJsonDsl {
  this["grade"] = Grades.Excellent
}.json(true)

{
  "grade": 7.0
}

# Implementing a simple query for Elasticsearch

We're going to implement a Kotlin DSL to implement this simple query

```json
{
  "query": {
    "term": {
      "myField": {
        "value": "some value",
        "boost": 2.0
      }
    }
  }
}
```

What's happening here:

- Elasticsearch queries are a json object that can have, amongst others a `query` property that takes any of the gazillions of different queries that it supports.
- We only support the `term` query here. This is one of the simplest queries that allows you to match a field with a value.
- Note how Elasticsearch wraps the term query object with an objec that has a single property called `term`. This is slightly annoying to model
- And note how the term query configuration with the `value` and the `boost` is assigned to a property with our field name. Also not ideal.

First we define a [DSL marker](https://kotlinlang.org/api/latest/jvm/stdlib/kotlin/-dsl-marker/).

> Classes that define annotations marked with the @DslMarker annotation are used to define DSLs. These annotations are used to mark classes and receivers, preventing receivers marked with the same annotation to be accessed inside one another.

In [18]:
// using DslMarkers is useful with
// complicated DSLs
@DslMarker
annotation class SearchDSLMarker

Elasticsearch has many query types so we need a base class for our term query

In [19]:
@SearchDSLMarker
open class ESQuery(
  val name: String,
  val queryDetails: JsonDsl = JsonDsl()
)  {

  // Elasticsearch wraps everything in an outer object
  // with the name as its only key
  fun wrapWithName() = withJsonDsl() {
    this[name] = queryDetails
  }
}

Next we implement a QueryDsl to model the outer part of the query.

In [20]:
// We'll use this to add extension functions for specific query types
interface QueryClauses

// abbreviated version of the
// Elasticsearch Query DSL in kt-search
class QueryDsl:
  JsonDsl(namingConvention = PropertyNamingConvention.ConvertToSnakeCase),
  // helper interface that we define
  // extension functions on
  QueryClauses
{
  // Elasticsearch often wraps objects in
  // another object. So we use a custom
  // setter here to hide that.
  var query: ESQuery
    get() {
      val map =
        this["query"] as Map<String, JsonDsl>
      val (name, details) = map.entries.first()
      // reconstruct the ESQuery
      return ESQuery(name, details)
    }
    set(value) {
      // queries extend ESQuery
      // which takes care of the wrapping
      // via wrapWithName
      this["query"] = value.wrapWithName()
    }
}

// easy way to create a query
fun query(block: QueryDsl.()->Unit): QueryDsl {
  return QueryDsl().apply(block)
}


Now we can implement our TermQuery. It needs a `TermQueryConfig` that we'll assign

In [21]:
// configuration for term queries
// this is a subset of the supported
// properties.
class TermQueryConfig : JsonDsl() {
  var value by property<String>()
  var boost by property<Double>()
}

// the dsl class for creating term queries
// this is one of the most basic queries
// in elasticsearch
@SearchDSLMarker
class TermQuery(
  field: String,
  value: String,
  termQueryConfig: TermQueryConfig = TermQueryConfig(),
  block: (TermQueryConfig.() -> Unit)? = null
) : ESQuery("term") {
  // on init, apply the block to the configuration and
  // assign it in the queryDetails from the parent
  init {
    queryDetails.put(field, termQueryConfig, PropertyNamingConvention.AsIs)
    termQueryConfig.value = value
    block?.invoke(termQueryConfig)
  }
}

Finally we add some extension functions to make life easier for our users. 
They'll be able to auto completer term inside a `QueryDsl` block because it implements `QueryClauses`

In [22]:
fun QueryClauses.term(
  field: String,
  value: String,
  block: (TermQueryConfig.() -> Unit)? = null
) =
  TermQuery(field, value, block = block)



And of course our users will likely want to use data classes to model their Json documents in Elasticsearch so wouldn't it be nice if they could use **property references**?

In [23]:
fun QueryClauses.term(
  field: KProperty<*>,
  value: String,
  block: (TermQueryConfig.() -> Unit)? = null
) = TermQuery(field.name, value, block = block)

## Using the QueryDsl

Creating the DSL is a bit of work but it helps our users by making things easy for them.

In [24]:
data class MyModelClassInES(val myField: String)

val q = query {
  query = term(MyModelClassInES::myField, "some value")
}

q.json(true)

{
  "query": {
    "term": {
      "myField": {
        "value": "some value"
      }
    }
  }
}

# Let's use our shiny new DSL with Opensearch

We'll use Java's HTTP client to talk to Opensearch. I have it running locally on port 9200. 

```shell
docker run -d --rm -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "DISABLE_SECURITY_PLUGIN=true" opensearchproject/opensearch:latest
```

Note, I've configured it without a volume and `--rm` so data will be gone when you stop the docker container. Refer to the documentation for more info on how to run this properly.



In [46]:
// we need some more imports
import java.net.URI
import java.net.http.*
import java.net.http.HttpResponse.BodyHandlers
import java.net.http.HttpRequest.BodyPublishers

val baseUrl = "http://localhost:9200"


We need some content in Elasticsearch

In [50]:
// clean up the old index
HttpClient.newHttpClient().send(
    HttpRequest.newBuilder()
        .uri(URI.create("$baseUrl/jsondsldemo"))
        .header("Content-Type", "application/json")
        .DELETE()
        .build(), 
    BodyHandlers.ofString())

(DELETE http://localhost:9200/jsondsldemo) 404

Let's create some documents in Elasticsearch. Since Elasticsearch is a JSON document store, of course we use json-dsl ...

In [60]:
class TestDoc() : JsonDsl(namingConvention = PropertyNamingConvention.AsIs) {
    var id by property<Int>()
    var title by property<String>()
    var myField by property<List<String>>()
}

fun testDoc(block: TestDoc.() -> Unit) = TestDoc().apply(block)
val docs = listOf(
    testDoc {
        id = 1
        title = "There can only be foo"
        myField = listOf("foo")
    },    
    testDoc {
        id = 2
        title = "Also bar"
        myField = listOf("foo","bar")
    },    
    testDoc {
        id = 3
        title = "It's foobarred"
        myField = listOf("foo","foobar")
    },    
    testDoc {
        id = 4
        title = "No foo"
        myField = listOf("bar")
    },    
)
docs.forEach { document ->
HttpClient.newHttpClient().send(
    HttpRequest.newBuilder()
        .uri(URI.create("$baseUrl/jsondsldemo/_doc/${document["id"]}"))
        .header("Content-Type", "application/json")
        .PUT(BodyPublishers.ofString(
            // convert out docs to json
            document.json()
        ))
        .build(), 
    BodyHandlers.ofString())
}

In [61]:

val response = HttpClient.newHttpClient().send(
    HttpRequest.newBuilder()
        .uri(URI.create("$baseUrl/jsondsldemo/_search"))
        .header("Content-Type", "application/json")
        .POST(BodyPublishers.ofString(
            query {
                
            }.json()
        ))
        .build(), 
    BodyHandlers.ofString())

println("Status: ${response.statusCode()}, Body: ${response.body()}")

Status: 200, Body: {"took":660,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"jsondsldemo","_id":"1","_score":1.0,"_source":{"id":1,"title":"There can only be foo","myField":["foo"]}},{"_index":"jsondsldemo","_id":"2","_score":1.0,"_source":{"id":2,"title":"Also bar","myField":["foo","bar"]}},{"_index":"jsondsldemo","_id":"3","_score":1.0,"_source":{"id":3,"title":"It's foobarred","myField":["foo","foobar"]}},{"_index":"jsondsldemo","_id":"4","_score":1.0,"_source":{"id":4,"title":"No foo","myField":["bar"]}}]}}
